In [5]:
import os
import gc
import shutil
if os.getenv("COLAB_RELEASE_TAG"):
    # Colab
    print("RUNING ON COLAB")
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = "/content/drive/MyDrive/GaggleSignLang/asl-signs"
    WORKING_DIR = "/content/asl-work"
    ARCHIVE_DIR = "/content/drive/MyDrive/GaggleSignLang"
    MODEL_DIR = "/content/drive/MyDrive/GaggleSignLang/models"
elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
    # KAGGLE
    print("RUNNING ON KAGGLE")
    BASE_DIR = "/kaggle/input/asl-signs"
    WORKING_DIR = "/kaggle/working"
    ARCHIVE_DIR = "/kaggle/working"
    MODEL_DIR  = "/kaggle/working"
else: 
    # Jupiter  MacOS
    print("RUNING JUPITER LOCAL")
    BASE_DIR = "/Users/johnhanratty/ASLtest/asl-signs"  #"/Users/johnhanratty/ASLtest/asl-signs"
    WORKING_DIR = "/Users/johnhanratty/ASLtest"
    ARCHIVE_DIR = "/Users/johnhanratty/ASLtest"
    MODEL_DIR = "/Users/johnhanratty/ASLtest/models"

import time


import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
from random import seed, sample

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

import warnings
warnings.filterwarnings(action='ignore')

LANDMARK_FILES_DIR = f'{BASE_DIR}/train_landmark_files'
TRAIN_FILE = f"{BASE_DIR}/train.csv"

FRAMES_OUT = 32 # 16
PTS_IN_FRAME = 345
DIMC = [0,1,2]
DIMS = len(DIMC)
WORKERS = 0   # dataoader work var  0 for MAC, 4 for online

print('done')

RUNING JUPITER LOCAL
done


# Create Training Data

## Read Parquet Files (provided)
Routine provide by contest to read parquet file containing a video (i.e. landmarks of video) of one sign frame sequence.  Each video can contain 8-500+ frames with different frame rates (frames per second).

In [6]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

# Video Preprocessing Model (FeatureGen)
The FeatureGen() model recieves one video from the load_relevant_data_subset() and preprocesses it for machine learning. This model was initially written in Numpy but convered to Pytorch tensors to aid the Pytorch to Onnx to tflite conversion.   

In [7]:
# CNN TORCH FEATUREGEN MODEL 
ROWS_PER_FRAME = 543  # combined face, lefth, pose, righth
PR_PTS = [40, 44, 48, 52, 56, 60, 43, 46, 50, 54, 58]
SC_PTS = [40, 98, 102, 106, 110, 114, 97, 102, 106, 110, 114]
PO_PTS = [60, 73, 80, 81, 76, 77, 68, 69, 70, 71, 75, 74]
LI_PTS = [5, 0, 4, 8, 12, 16, 20, 24, 28, 32, 36]
PR_LEN = len(PR_PTS) * DIMS * 5  # 5 = number of aggregations e.g. max,min
SC_LEN = len(SC_PTS) * DIMS * 5
PO_LEN = len(PO_PTS) * DIMS * 5
LI_LEN = len(LI_PTS) * DIMS * 5
CNN_FEAT_LEN = PR_LEN + SC_LEN + PO_LEN + LI_LEN 

print("Feature Len", CNN_FEAT_LEN, PR_LEN, SC_LEN, PO_LEN, LI_LEN)

# FILTER FEATURES IN EACH FRAME  - FACE, POSE & HANDs
class FeatureGen(nn.Module):
    def __init__(self):
        super(FeatureGen, self).__init__()
        pass
    
    def forward(self, x):
        x = torch.tensor(x)
        
        # FILTER TO SPECIFIED FRAMES (FRAMES_OUT)
        seed(24)
        n_frames = x.size()[0]
        # Trim to # of frames to FRAMES_OUT
        if n_frames > FRAMES_OUT:
            idx = sorted((sample(range(0, n_frames), FRAMES_OUT)))
            x=x[idx]
        n_frames = x.size()[0]
        # FLATTENING ROWS BY TYPE and CONCATENATING TO ONE ROW PER FRAME 3D (XYZ)
        # INPUT NUMPY, TORCH OUTPUT

        # The Video contains n_frames each containing exactly ROWS_PER_FRAME (543)frames.
        # The frames in each are in order of feature type.
        # The rows conain x, y, z for a feature
        #   Video Format = [n_frames][543 frames][3 xyz coordinates]
        
        # Create views by data type (e.g. one point on hand) 
        # by selecting rows for each frame
        # face_x = x[:,:468,:].contiguous().view(-1, 468*3)
        lips_idx = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324, 146, 91, 181, 84, 17, 314, 405, 321, 375]
        lips_x = x[:, lips_idx,:].contiguous().view(-1, len(lips_idx)*3)
        lefth_x = x[:,468:489,:].contiguous().view(-1, 21*3)
        pose_x = x[:,489:522,:].contiguous().view(-1, 33*3)
        righth_x = x[:,522:,:].contiguous().view(-1, 21*3)

        # Check for primary hand and if left hand swap and rotate hands
        if torch.isnan(lefth_x).sum() < torch.isnan(righth_x).sum():
            prime_x = lefth_x
            second_x = righth_x
        else:
            prime_x = righth_x.reshape(righth_x.size()[0], -1, DIMS)
            prime_x[:,:,0] = torch.from_numpy(np.subtract(np.nanmax(prime_x[:,:,0].numpy(), axis=1).reshape(-1,1),
                                    prime_x[:, :, 0].numpy()))
            prime_x = prime_x.reshape(prime_x.size()[0],-1)
            
            second_x = lefth_x.reshape(lefth_x.size()[0], -1, DIMS)
            second_x[:,:,0] = torch.from_numpy(np.subtract(np.nanmax(second_x[:,:,0].numpy(), axis=1).reshape(-1,1),
                                          second_x[:, :, 0].numpy()))
            second_x = second_x.reshape(second_x.size()[0],-1)
            
        
        # create video withfixed number of frames (FRAMES_OUT)
        # initialize with NoN so later operations can ignore them (e.g. nanmean()) 
        xfeat = torch.full([FRAMES_OUT, PTS_IN_FRAME], np.nan)
        
        # center frames
        offset = (FRAMES_OUT - n_frames) // 2  # center frames in output data in each frame in video
        
        # flatten types into one row per frame
        xfeat[offset:n_frames+offset,:] = torch.cat([lips_x, prime_x, pose_x, second_x], axis=1)  # concatenate types
        
        ############# CNN Specific
        # function aggregates frames in one row
        # which contains mean, median, max, min and var of frames
        def distDiff(ds, ref, pts):
            ds = ds.reshape(ds.size()[0],  -1, DIMS)
            d = torch.hstack([torch.from_numpy(np.nanmean(ds[:, pts, :].numpy(), axis=0)), 
                           torch.from_numpy( np.nanmedian(ds[:, pts, :].numpy(), axis=0)), 
                           torch.from_numpy(np.nanmax(ds[:, pts, :].numpy(), axis=0)), 
                           torch.from_numpy(np.nanmin(ds[:, pts, :].numpy(), axis=0)),
                           torch.from_numpy(np.nanvar(ds[:, pts, :].numpy(), axis=0))
                           ]) 
            d = d.reshape(1, -1) 
            # NORMALIZE
            # d = (d - np.nanmean(d, keepdims=True)) / np.nanstd(d, keepdims=True) # -1 to 1
            # Replace NaN with zeros
            d = torch.where(torch.isnan(d), torch.tensor(0.0, dtype=torch.float32), d)
            return d
        
        # 
        d1 = distDiff(xfeat, 40, PR_PTS)
        d2 = distDiff(xfeat, 40, SC_PTS)
        d3 = distDiff(xfeat, 60, PO_PTS)
        d4 = distDiff(xfeat, 5,  LI_PTS)
        
        return torch.cat([d1,d2,d3,d4], axis=1)
    


Feature Len 675 165 165 180 165


In [8]:
## PROCESS EACH ROW (ONE PARQUET PER ROW)
def convert_row(row):
    x = load_relevant_data_subset(os.path.join(BASE_DIR, row[1].path))
    x = feature_converter(torch.tensor(x))
    return x, row[1].label

## LOOP THROUGH PARQUET FILES LISTED IN TRAIN FILE
##  SAVE RESULTS 
def convert_and_save_data():
    label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))
    df = pd.read_csv(TRAIN_FILE)
    df['label'] = df['sign'].map(label_map)
    
    print("Convert&Save", df.shape)
    #### FOR TESTING #################
    #df = df[0:20]
    ##################################

    npdata = np.zeros((df.shape[0], CNN_FEAT_LEN))  

    nplabels = np.zeros(df.shape[0])
    
    results = map(convert_row, df.iterrows())
    for i, (x,y) in tqdm(enumerate(results), total=df.shape[0]):
            npdata[i,:] = x
            nplabels[i] = y
    return npdata, nplabels
 

feature_converter = FeatureGen()
datax, datay = convert_and_save_data()



Convert&Save (94477, 5)


100%|████████████████████████████████████| 94477/94477 [13:36<00:00, 115.73it/s]


In [9]:
# Save dataset
np.save(f"{ARCHIVE_DIR}/cnn_data{FRAMES_OUT}.npy", datax)
np.save(f"{ARCHIVE_DIR}/cnn_labels.npy", datay)
 

In [10]:
# Load dataset
# Load dataset
datax = np.load(f"{ARCHIVE_DIR}/cnn_data{FRAMES_OUT}.npy")
datay = np.load(f"{ARCHIVE_DIR}/cnn_labels.npy") 



In [11]:
#MODEL
### NEW SEPARATED INPUTS
class ASLData(Dataset):
    def __init__(self,datax,datay):
        self.datax = datax
        self.datay = datay

    def __getitem__(self, index):
        return self.datax[index, :], self.datay[index]

    def __len__(self):
        return len(self.datay)

# https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab
class ASLModel(nn.Module):
    def __init__(self, p):
        super(ASLModel, self).__init__()
        
        # DATA in [1, CNN_FEAT_LEN] per video
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p)
        self.relu = nn.ReLU()

        L1OUT = 512  #1024 was ok
        L2OUT = 512

        self.layer_ph = nn.Linear(PR_LEN, L1OUT)
        self.batchnorm_ph = nn.BatchNorm1d(L1OUT)
        
        self.layer_sh = nn.Linear(SC_LEN, L1OUT)
        self.batchnorm_sh = nn.BatchNorm1d(L1OUT)
 
        self.layer_po = nn.Linear(PO_LEN, L1OUT)
        self.batchnorm_po = nn.BatchNorm1d(L1OUT)
 
        self.layer_li = nn.Linear(LI_LEN, L1OUT)
        self.batchnorm_li = nn.BatchNorm1d(L1OUT) 
 
        self.layer1 = nn.Linear(4*L1OUT, L2OUT)
        self.batchnorm1 = nn.BatchNorm1d(L2OUT)

        self.layerFC = nn.Linear(L2OUT, 250)
        self.softmax = nn.Softmax()
 
        
    def forward(self, x):
        phand = x[:,0:PR_LEN]            
        shand = x[:,PR_LEN:(PR_LEN+PR_LEN)]
        pose =  x[:,(PR_LEN+PR_LEN):(PR_LEN+PR_LEN+PO_LEN)]  
        lips =  x[:,(PR_LEN+PR_LEN+PO_LEN):CNN_FEAT_LEN]  
        
        ph = self.flatten(torch.tensor(phand).float()) 
        ph = self.layer_ph(ph)
        ph = self.batchnorm_ph(ph)
        ph = self.relu(ph)
        ph = self.dropout(ph)

        sh = self.flatten(torch.tensor(shand).float())       
        sh = self.layer_sh(sh)
        sh = self.batchnorm_sh(sh)
        sh = self.relu(sh)
        sh = self.dropout(sh)
       
        po = self.flatten(torch.tensor(pose).float())       
        po = self.layer_po(po)
        po = self.batchnorm_po(po)
        po = self.relu(po)
        po = self.dropout(po)
        
        li = self.flatten(torch.tensor(lips).float())       
        li = self.layer_li(li)
        li = self.batchnorm_li(li)
        li = self.relu(li)
        li = self.dropout(li)

        x = torch.cat((ph.view(ph.size(0), -1),
                       sh.view(sh.size(0), -1),
                       po.view(po.size(0), -1),
                       li.view(li.size(0), -1)), dim=1)
        # x = self.batchnorm0(x)
        x = self.layer1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layerFC(x)
       # x = self.softmax(x)

        return x


In [14]:
  PATH = f"{MODEL_DIR}/model_ccn{FRAMES_OUT}.sd"
torch.save(model.state_dict(), PATH)

PATH = f"{MODEL_DIR}/model_ccn{FRAMES_OUT}.pt"
torch.save(model, PATH)

# Save Pred and Perf
x = testx.detach().numpy()
prob_cnn = model(torch.tensor(x)).detach().numpy()

pred_list['cnn'] = np.argmax(prob_cnn, axis=1)
print(pred_list.head())

with open(f"{ARCHIVE_DIR}/pred_cnn{FRAMES_OUT}.pkl", 'wb') as f1:
       pickle.dump(pred_list, f1)
with open(f"{ARCHIVE_DIR}/prob_cnn{FRAMES_OUT}.pkl", 'wb') as f1:
       pickle.dump(prob_cnn, f1)
print(prob_cnn.shape)


   truth  cnn
0  206.0   78
1   20.0   96
2  178.0  103
3  114.0  114
4  221.0  221
(14172, 250)


In [15]:
## MULTI TRAINING
# !!! TRAINING DOES NOT RUN ON MAC OS - (cuda)
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("++++using GPU++++")
else:
  device = torch.device("cpu")
  print("++++using CPU++++")

EPOCHS = 21
BATCH_SIZE = 64
start_time = time.perf_counter()

#datax = datax.reshape(datax.shape[0],datax.shape[1], -1) #.swapaxes(1,2)
#datax = torch.tensor(datax)  # Convert to Torch Tensor
datax = torch.tensor(datax)  # Convert to Torch Tensor

trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.15, random_state=42)

# init list for saving predictions for ensemble processing
pred_list = pd.DataFrame(testy, columns=["truth"])

train_data = ASLData(trainx, trainy)
valid_data = ASLData(testx, testy)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=WORKERS, shuffle=True)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, num_workers=WORKERS, shuffle=False)
model = ASLModel(0.2).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
sched = torch.optim.lr_scheduler.StepLR(opt, step_size=300, gamma=0.95)
for i in range(EPOCHS):
    model.train()
    
    train_loss_sum = 0.
    train_correct = 0
    train_total = 0
    train_bar = train_loader
    for x,y in train_bar:
        x = torch.Tensor(x).float().to(device)
        y = torch.Tensor(y).long().to(device) 
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        train_loss_sum += loss.item()
        train_correct += np.sum((np.argmax(y_pred.detach().cpu().numpy(), axis=1) == y.cpu().numpy()))
        train_total += 1
        sched.step()
        
    val_loss_sum = 0.
    val_correct = 0
    val_total = 0
    model.eval()
    for x,y in val_loader:
        x = torch.Tensor(x).float().to(device)
        y = torch.Tensor(y).long().to(device)
        
        with torch.no_grad():
            y_pred = model(x)
            loss = criterion(y_pred, y)
            val_loss_sum += loss.item()
            val_correct += np.sum((np.argmax(y_pred.cpu().numpy(), axis=1) == y.cpu().numpy()))
            val_total += 1
    print(f"DIM={DIMS} FRAMES={FRAMES_OUT}, FEAT={PTS_IN_FRAME}")                          
    print(f"Epoch:{i} > Train Loss: {(train_loss_sum/train_total):.04f}, Train Acc: {train_correct/len(train_data):0.04f}")
    print(f"Epoch:{i} > Val Loss: {(val_loss_sum/val_total):.04f}, Val Acc: {val_correct/len(valid_data):0.04f}")
    print("="*50)

# Save the pytorch model
PATH = f"{MODEL_DIR}/model_ccn{FRAMES_OUT}.sd"
torch.save(model.state_dict(), PATH)

PATH = f"{MODEL_DIR}/model_ccn{FRAMES_OUT}.pt"
torch.save(model, PATH)

# Save Pred and Perf
x = testx.detach().numpy()
prob_cnn = model(torch.tensor(x)).detach().numpy()

pred_list['cnn'] = np.argmax(prob_cnn, axis=1)
print(pred_list.head())

with open(f"{ARCHIVE_DIR}/pred_cnn{FRAMES_OUT}.pkl", 'wb') as f1:
       pickle.dump(pred_list, f1)
with open(f"{ARCHIVE_DIR}/prob_cnn{FRAMES_OUT}.pkl", 'wb') as f1:
       pickle.dump(prob_cnn, f1)
print(prob_cnn.shape)

print("Accuracy:", np.mean(pred_list.truth == pred_list.cnn))
print("#### ELAPSED TIME:", time.perf_counter()-start_time)



++++using CPU++++
DIM=3 FRAMES=32, FEAT=345
Epoch:0 > Train Loss: 3.4987, Train Acc: 0.2172
Epoch:0 > Val Loss: 3.0151, Val Acc: 0.2730
DIM=3 FRAMES=32, FEAT=345
Epoch:1 > Train Loss: 2.3660, Train Acc: 0.4130
Epoch:1 > Val Loss: 2.4597, Val Acc: 0.4073
DIM=3 FRAMES=32, FEAT=345
Epoch:2 > Train Loss: 2.0093, Train Acc: 0.4924
Epoch:2 > Val Loss: 1.8661, Val Acc: 0.5386
DIM=3 FRAMES=32, FEAT=345
Epoch:3 > Train Loss: 1.7829, Train Acc: 0.5446
Epoch:3 > Val Loss: 1.8927, Val Acc: 0.5210
DIM=3 FRAMES=32, FEAT=345
Epoch:4 > Train Loss: 1.6201, Train Acc: 0.5777
Epoch:4 > Val Loss: 1.5717, Val Acc: 0.6154
DIM=3 FRAMES=32, FEAT=345
Epoch:5 > Train Loss: 1.4902, Train Acc: 0.6105
Epoch:5 > Val Loss: 1.5503, Val Acc: 0.6134
DIM=3 FRAMES=32, FEAT=345
Epoch:6 > Train Loss: 1.3824, Train Acc: 0.6349
Epoch:6 > Val Loss: 1.4339, Val Acc: 0.6494
DIM=3 FRAMES=32, FEAT=345
Epoch:7 > Train Loss: 1.3005, Train Acc: 0.6564
Epoch:7 > Val Loss: 1.3642, Val Acc: 0.6674
DIM=3 FRAMES=32, FEAT=345
Epoch:8 > Tr

In [ ]:
# ==================================================
# DIM=3 FRAMES=32, FEAT=345
# Epoch:21 > Train Loss: 0.9673, Train Acc: 0.7361
# Epoch:21 > Val Loss: 1.2287, Val Acc: 0.7047
# ==================================================

# ========Full Data - no add Norm, Centered ========
# DIM=3 FRAMES=32, FEAT=345
# Epoch:38 > Train Loss: 0.9867, Train Acc: 0.7321
# Epoch:38 > Val Loss: 1.2608, Val Acc: 0.6988
# ==================================================

# ========Full Data - no additional Normalizeation/Centering=====================================
# DIM=3 FRAMES=32, FEAT=345
# Epoch:39 > Train Loss: 1.4974, Train Acc: 0.6059
# Epoch:39 > Val Loss: 1.7298, Val Acc: 0.5839
# ==================================================

# =====Full no reverse, centered =============================================
# DIM=3 FRAMES=32, FEAT=345
# Epoch:39 > Train Loss: 1.2104, Train Acc: 0.6745
# Epoch:39 > Val Loss: 1.4204, Val Acc: 0.6515
# ==================================================
# Additional Normalize and Nan Step  0.6100

# Torch Single Model Save, Retrieve and Rum

In [43]:
## SAVE MODEL

PATH = f"{MODEL_DIR}/modeltccn{FRAMES_OUT}flat.pt"
torch.save(model, PATH)



In [44]:
##LOAD MODEL

PATH = f"{MODEL_DIR}/modeltccn{FRAMES_OUT}flat.pt"
mod = torch.load(PATH)


In [45]:
# SAVE PYTORCH MODELS
class AModel(nn.Module):
    def __init__(self):
        super(AModel, self).__init__()
        
        self.InputFormat = FeatureGen() #feature_converter
        self.InferModel = mod
        self.InferModel.eval()
    
    def forward(self, x):
        x = self.InputFormat(x)
        pred = self.InferModel(x)
        return pred

mod_cnn = AModel()
mod_cnn_pt_path = f"{MODEL_DIR}/modelcnn{FRAMES_OUT}test.pt"
torch.save(mod_cnn, mod_cnn_pt_path)



In [46]:
# Run merged PyTorch Model
label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))
df = pd.read_csv(TRAIN_FILE)
df['label'] = df['sign'].map(label_map)

mod_cnn_pt_path = f"{MODEL_DIR}/modelcnn{FRAMES_OUT}test.pt"
Infmodel = torch.load(mod_cnn_pt_path)

d = df[2022:2023]

x = load_relevant_data_subset(os.path.join(BASE_DIR, d['path'].item()))
pred = Infmodel(x)

print("truth:", d.label, d.sign, "prediction=", np.argmax(pred.detach().numpy()))


truth: 2022    56
Name: label, dtype: int64 2022    dance
Name: sign, dtype: object prediction= 56


# TFLITE CONVERT

In [49]:
!pip install tensorflow --quiet --root-user-action=ignore
!pip install tensorflow_probability --quiet --root-user-action=ignore

!pip install onnx-tf --quiet --root-user-action=ignore
!pip install tflite-runtime  --quiet --root-user-action=ignore
import onnx_tf
import tflite_runtime
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

2023-04-30 09:30:43.203079: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [61]:
## CNN MODEL CONVERSION
# pmodel_path = f"{ARCHIVE_DIR}/models/modelccn16flat.sd"
# modelc = ASLModel(.1)
# modelc.load_state_dict(torch.load(py_model_path))
# modelc.eval()

PATH = f"{MODEL_DIR}/modeltccn{FRAMES_OUT}flat.pt"
mod = torch.load(PATH)


sample_input = torch.rand((64, 615)).float()
onnx_mod_cnn_path = f"{MODEL_DIR}/model_cnn.onnx"

torch.onnx.export(
    mod,                    # PyTorch Model
    sample_input,             # Input tensor
    onnx_mod_cnn_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,         # Operator support version
    input_names=['input'],     # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={'input' : {0: 'input'}
    }
)
onnx_mod_cnn_gen = onnx.load(onnx_mod_cnn_path)
tf_rep = prepare(onnx_mod_cnn_gen)

tf_mod_cnn_path = 'f"{MODEL_DIR}/tf_mod_cnn'
tf_rep.export_graph(tf_mod_cnn_path)



================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



2023-04-30 09:50:42.993522: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input' with dtype float and shape [?,615]
	 [[{{node input}}]]
2023-04-30 09:50:42.993602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '10241' with dtype float and shape [512]
	 [[{{node 10241}}]]
2023-04-30 09:50:42.993649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '10253' with dtype float and shape [512]
	 [[{{node 10253}}]]
2023-04-30 09:50:42.993717

INFO:tensorflow:Assets written to: f"{MODEL_DIR}/tf_mod_cnn/assets


INFO:tensorflow:Assets written to: f"{MODEL_DIR}/tf_mod_cnn/assets


In [62]:
## FEAT_GEN MODEL CONVERSION
# pmodel_path = f"{ARCHIVE_DIR}/models/modelccn16flat.sd"
# modelc = ASLModel(.1)
# modelc.load_state_dict(torch.load(py_model_path))
# modelc.eval()

gen_feat = FeatureGen()

sample_input = torch.rand((50, 543, 3))
onnx_gen_feat_path = f"{MODEL_DIR}/gen_featt.onnx"

torch.onnx.export(
    gen_feat,                    # PyTorch Model
    sample_input,             # Input tensor
    onnx_gen_feat_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,         # Operator support version
    input_names=['input'],     # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={'input' : {0: 'input'}
    }
)
onnx_feat_gen = onnx.load(onnx_gen_feat_path)
tf_rep = prepare(onnx_feat_gen)

tf_gen_feat_path = 'f"{MODEL_DIR}/tf_gen_feat'
tf_rep.export_graph(tf_gen_feat_path)



================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

INFO:tensorflow:Assets written to: f"{MODEL_DIR}/tf_gen_feat/assets


INFO:tensorflow:Assets written to: f"{MODEL_DIR}/tf_gen_feat/assets


In [63]:
import tensorflow as tf

class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        self.feature_gen = tf.saved_model.load(tf_gen_feat_path)
        self.model = tf.saved_model.load(tf_mod_cnn_path)
        self.feature_gen.trainable = False
        self.model.trainable = False
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
    ])
    def call(self, input):
        output_tensors = {}
        features = self.feature_gen(**{'input': input})['output']
        output_tensors['outputs'] = self.model(**{'input': tf.expand_dims(features, 0)})['output'][0,:]
        return output_tensors
    
    
mytfmodel = ASLInferModel()
INFER_PATH=f'{MODEL_DIR}/tf_infer_model', 
tf.saved_model.save(mytfmodel, 
                    INFER_PATH, 
                    signatures={'serving_default': mytfmodel.call})


ValueError: in user code:

    File "/var/folders/p5/bqxhgw2x5bgfr_7ndb9qv2z00000gn/T/ipykernel_85568/180413784.py", line 16, in call  *
        features = self.feature_gen(**{'input': input})['output']

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (0 total):
        * 
      Keyword arguments: {'input': <tf.Tensor 'input:0' shape=(None, 543, 3) dtype=float32>}
    
     Expected these arguments to match one of the following 1 option(s):
    
    Option 1:
      Positional arguments (0 total):
        * 
      Keyword arguments: {}


In [ ]:
tf.saved_model.save(model, save_model_path) # save as savemodel form
test_model = tf.keras.models.load_model(save_model_path, custom_objects={"TFBertModel": transformers.TFBertModel}) # load model and point out the custom_objects

In [29]:
import tensorflow as tf

class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        tf_mod_cnn_path = 'f"{MODEL_DIR}/tf_mod_cnn'

        self.model = tf.saved_model.load(tf_mod_cnn_path)
        self.model.trainable = False
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
    ])
    def call(self, input):
        output_tensors = {}
        print("INSIDE ")
        output_tensors['outputs'] = self.model(**{'input': input})
        print("2")
        return output_tensors
    
    
mytfmodel = ASLInferModel()
print("ok")
tf.saved_model.save(mytfmodel, 
                    f'{MODEL_DIR}/tf_infer_model', 
                    signatures={'serving_default': mytfmodel.call})



ok
INSIDE 


ValueError: in user code:

    File "/var/folders/p5/bqxhgw2x5bgfr_7ndb9qv2z00000gn/T/ipykernel_82561/3126969839.py", line 17, in call  *
        output_tensors['outputs'] = self.model(**{'input': input})

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (0 total):
        * 
      Keyword arguments: {'input': <tf.Tensor 'input:0' shape=(None, 543, 3) dtype=float32>}
    
     Expected these arguments to match one of the following 1 option(s):
    
    Option 1:
      Positional arguments (0 total):
        * 
      Keyword arguments: {}


In [ ]:
# CONVERT MODEL TO TFLITE
converter = tf.lite.TFLiteConverter.from_saved_model(tf_mod_cnn_path) # path to the SavedModel directory
tflite_model = converter.convert()
# Save the model.
TF_MODEL_PATH_LITE = f"{MODEL_DIR}/model_cnn.tflite"
with open(TF_MODEL_PATH, 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=TF_MODEL_PATH_LITE)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())
print("found_signatures")
print(found_signatures[0])

prediction_fn = interpreter.get_signature_runner("serving_default")

label_map = json.load(open(f"{BASE_DIR}/sign_to_prediction_index_map.json", "r"))   ###CHANGE
df = pd.read_csv(TRAIN_FILE)
df['label'] = df['sign'].map(label_map)

vid = df.iloc[26].path
x = load_relevant_data_subset(f"{BASE_DIR}/{vid}")
output = prediction_fn(x)
#interpreter.invoke()

sign = np.argmax(output["outputs"])

print(sign)